In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install huggingface_hub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.


In [2]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = getattr(torch, "float16"), bnb_4bit_quant_type = "nf4"))
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 100)
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = load_dataset(path = "vivekdugale/llama2_filtered_dataset_amod_helios_450", split = "train"),
                               tokenizer = llama_tokenizer,
                               peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 64, lora_alpha = 16, lora_dropout = 0.1),
                               dataset_text_field = "text")

llama_sft_trainer.train()



2024-03-23 08:49:11.361232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 08:49:11.361394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 08:49:11.498253: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=100, training_loss=2.216045379638672, metrics={'train_runtime': 1357.3422, 'train_samples_per_second': 0.589, 'train_steps_per_second': 0.074, 'total_flos': 7389332827914240.0, 'train_loss': 2.216045379638672, 'epoch': 1.75})

In [4]:
user_prompt = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues but I never get around to it. How can I change my feeling of being worthless to everyone"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 1000)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues but I never get around to it. How can I change my feeling of being worthless to everyone [/INST]  It's important to recognize that you are not alone in feeling this way. Many people struggle with feelings of worthlessness and low self-esteem, and it can be a very difficult and painful experience. However, it's important to know that these feelings are not a reflection of your true worth as a person. You are not worthless, and you are not alone.

It's important to reach out for support and to talk to someone you trust about how you are feeling. This can be a friend, family member, or mental health professional. Talking about your feelings can help you to process them and to gain a new perspective on yourself and your life.

It can also be helpful to

In [5]:
user_prompt = "I don't feel comfortable to share my feelings with anyone. I want you to help me, please! I'm at a terrible place mentally & at this rate, I don't think I can live"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 1000)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

<s>[INST] I don't feel comfortable to share my feelings with anyone. I want you to help me, please! I'm at a terrible place mentally & at this rate, I don't think I can live [/INST]  Sorry to hear that you're feeling uncomfortable sharing your feelings with anyone. It's completely understandable to feel that way, especially when it comes to sensitive topics like mental health. I'm here to listen and help in any way I can, without judgment or pressure.

It's important to remember that you don't have to go through this alone. There are many resources available to help you manage your mental health, including therapy, support groups, and hotlines. These options can provide a safe and confidential space for you to talk about your feelings and work through them with a trained professional.

If you're not ready to talk to someone in person, there are also many online resources available. You can find online therapy platforms, support groups, and hotlines that can provide a similar level of s

In [6]:
user_prompt = "I am depressed is what i feel. please help"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 1000)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

<s>[INST] I am depressed is what i feel. please help [/INST]  Sorry to hear that you're feeling depressed. It's important to know that you're not alone, and there are many resources available to help you manage your feelings and improve your mental health. Here are some suggestions:

1. Seek professional help: Consider talking to a mental health professional, such as a therapist or counselor, who can help you identify the underlying causes of your depression and develop strategies for managing it. You can find mental health professionals in your area through your insurance provider or online directories.
2. Practice self-care: Engage in activities that bring you joy and help you relax, such as exercise, meditation, or spending time with loved ones. Make sure to get enough sleep, eat a healthy diet, and avoid alcohol and drugs, which can worsen depression.
3. Connect with others: Reach out to friends, family, or support groups for help and encouragement. Talking to others who understand

In [7]:
user_prompt = "I'm feeling down at work nowadays, and no one values me. I'm at a difficult place & I'm feeling depressed"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 1000)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

<s>[INST] I'm feeling down at work nowadays, and no one values me. I'm at a difficult place & I'm feeling depressed [/INST]  Sorry to hear that you're feeling down and unvalued at work. It can be really tough to feel like you're not being recognized or appreciated for your efforts, and it can affect your mental health and overall well-being. Here are some suggestions that may help:

1. Talk to someone: It can be helpful to talk to someone you trust about how you're feeling. This could be a friend, family member, or a mental health professional. Talking through your feelings can help you process and understand them better, and it can also be a great way to get support and advice.
2. Practice self-care: Take care of yourself by doing things that make you feel good, such as exercise, meditation, or spending time with loved ones. Self-care can help you feel more grounded and centered, and it can also help you manage stress and anxiety.
3. Set boundaries: If you're feeling overwhelmed or un

In [8]:
user_prompt = "I am feeling consistently unhappy at work, but still I can't bring myself to change my job. I've tried to implement almost whatever you have advised, and I haven't gotten anywhere. Please help me!"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 1000)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

<s>[INST] I am feeling consistently unhappy at work, but still I can't bring myself to change my job. I've tried to implement almost whatever you have advised, and I haven't gotten anywhere. Please help me! [/INST]  Sorry to hear that you're feeling unhappy at work. It's understandable to feel stuck in a situation that's not fulfilling, but it's important to take action to make a change. Here are some additional suggestions that may help you:

1. Identify the source of your unhappiness: Take some time to reflect on what's causing your unhappiness at work. Is it the work itself, the people you're working with, or something else? Once you've identified the source, you can start working on a solution.
2. Set clear goals for yourself: Setting clear goals can help you stay motivated and focused on making a change. Write down what you want to achieve, and break it down into smaller, manageable steps.
3. Seek support from others: Talk to friends, family, or a therapist about how you're feelin

In [9]:
user_prompt = "I think the source of my unhappiness is the work itself. I joined this field thinking I would be able to expand my knowledge and follow my passion. But, the only thing I think of at work is of endless deadlines & constant stress."
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 1000)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

<s>[INST] I think the source of my unhappiness is the work itself. I joined this field thinking I would be able to expand my knowledge and follow my passion. But, the only thing I think of at work is of endless deadlines & constant stress. [/INST]  It sounds like you're feeling unfulfilled and stressed out in your job, which can be a challenging and frustrating experience. Here are some suggestions that may help you re-evaluate your priorities and find a more fulfilling career path:

1. Reflect on your values and passions: Take some time to think about what truly matters to you in a career. What are your core values and passions? What kind of work do you find most meaningful and fulfilling? Consider how your current job aligns with these values and passions, and whether there are other areas that might be a better fit.
2. Identify your strengths and skills: Think about what you're naturally good at and what skills you've developed over time. What are your strengths, and how can you use

In [11]:
user_prompt = "My career is related to software, but lately I feel like I've deviated from my line of work. As promotions have come by through the years the work has changed so much that I barely recognize what I do now. I don't know how to adapt and find my way back."
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 1000)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

<s>[INST] My career is related to software, but lately I feel like I've deviated from my line of work. As promotions have come by through the years the work has changed so much that I barely recognize what I do now. I don't know how to adapt and find my way back. [/INST]  It's common for professionals in the tech industry to feel like they've strayed from their original path, especially as technology and industries evolve. Here are some suggestions to help you find your way back to your original line of work:

1. Reflect on your values and interests: Take some time to reflect on what matters most to you in your work. What are your core values and interests? What kind of work environment do you thrive in? By understanding what drives you, you can identify areas that align with your original goals and motivations.
2. Identify the root cause of your disconnection: Understanding why you feel disconnected from your original line of work can help you address the root cause. Is it due to a la